In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib.image as mpimg

global estimated_capacity_charge_phase_in_Ah
global cumulative_charge_in_amperes_per_hour
global cumulative_Ah_discharge_phase

# Load and preprocess Battery charge and discharge cycle data
filename = 'VAH02.csv'
data_folder = 'battery_data'
# Full path to CSV file
filepath = os.path.join(data_folder, filename)

# Specify the number of rows of Data in the 'VAH02.csv' file to process (e.g., 6000)
num_rows_to_look_at = 14500        
# Read CSV into DataFrame
data = pd.read_csv(filepath)
# Read CSV into DataFrame
data = data.iloc[:num_rows_to_look_at]  # Keep only the first num_rows_to_look_at rows

# Extract time and current arrays from DataFrame
time_in_sec = data['time_s'].values
# Normalize time so it starts at zero
time_in_sec = time_in_sec - time_in_sec[0]
# Convert current from mA to A
current_in_amperes = data['I_mA'].values / 1000 

# Create Boolean mask for where current is positive (charging)
charge_mask = current_in_amperes > 0 
# Get Indices where charging happens            
charge_indices = np.where(charge_mask)[0]
# Get the start index of charging phase (this is the index where we have the first positive current) 
charge_phase_start_index = charge_indices[0]
# Get the end index of charging phase (this is the index where we have last positive current +1 for slicing)   
charge_phase_end_index = charge_indices[-1] + 1

# Extract slices of charging time and current
charge_phase_duration = time_in_sec[charge_phase_start_index:charge_phase_end_index]
charge_phase_current = current_in_amperes[charge_phase_start_index:charge_phase_end_index]

# Find index where discharge first starts (i.e., where current goes below zero)
discharge_phase_start_index = np.where(current_in_amperes < 0)[0][0]
# Extract discharge phase time begining from discharge_start_index
discharge_phase_duration_raw = time_in_sec[discharge_phase_start_index:]
# Extract discharge phase current starting from discharge_start_index
discharge_phase_current_raw = current_in_amperes[discharge_phase_start_index:] 

In [ ]:
def function_to_estimate_capacity_in_Ah_from_charging():
    global estimated_capacity_charge_phase_in_Ah
    global cumulative_charge_in_amperes_per_hour
    # Calculate time intervals between consecutive charge samples (in seconds). This would be used for numerical integration.
    dt_charging_duration = np.diff(charge_phase_duration) 
    # Calculate average current between consecutive samples using trapezoidal rule.This approximates current variation linearly over each interval.
    avg_current_charge = (charge_phase_current[:-1] + charge_phase_current[1:]) / 2 
    # Calculate the cumulative charge in amperes per second by multiplying average current by corresponding time intervals in seconds. The do a cumulative sum to get total charge delivered at each time point.
    cumulative_charge_in_amperes_per_sec = np.cumsum(avg_current_charge * dt_charging_duration)  
    # Convert the cumulative charge in amperes per second (As) to cumulative charge in amperes per hour (Ah)
    cumulative_charge_in_amperes_per_hour = cumulative_charge_in_amperes_per_sec / 3600 
    # Estimate battery capacity. This is the total charge delivered during charging phase (i.e., the last value).
    estimated_capacity_charge_phase_in_Ah = cumulative_charge_in_amperes_per_hour[-1]
    return estimated_capacity_charge_phase_in_Ah

In [ ]:

def function_to_estimate_capacity_in_Ah_from_discharging():
    global cumulative_Ah_discharge_phase
    # Calculate time intervals between consecutive discharge samples (in seconds). This would be used for numerical integration.
    dt_discharge_phase_duration = np.diff(discharge_phase_duration_raw)
    # Calculate average current between consecutive samples using trapezoidal rule.This approximates current variation linearly over each interval.
    avg_current_discharge_phase = (discharge_phase_current_raw[:-1] + discharge_phase_current_raw[1:]) / 2 
    # Calculate the cumulative discharge in amperes per second by multiplying average current by corresponding time intervals in seconds. Then do a cumulative sum to get total discharge taken at each time point.
    cumulative_discharge_in_amperes_per_sec = np.cumsum(avg_current_discharge_phase * dt_discharge_phase_duration)  
    # Convert the cumulative discharge in amperes per second (As) to cumulative discharge in amperes per hour (Ah)
    cumulative_Ah_discharge_phase = cumulative_discharge_in_amperes_per_sec / 3600
    return cumulative_Ah_discharge_phase

# Capacity Calculation
This is achieved by integrating current over time



In [ ]:

function_to_estimate_capacity_in_Ah_from_charging()

# Normalize cumulative charge using estimated capacity. This gives SOC as a fraction [0,1].
State_of_Charge_from_charging = cumulative_charge_in_amperes_per_hour / estimated_capacity_charge_phase_in_Ah
# Insert initial SOC of 0 at the start. This is under the assumption that battery was fully discharged before charging.
State_of_Charge_from_charging = np.insert(State_of_Charge_from_charging, 0, 0.0)  
# For the purpose of plotting alignment, ensure the length of time array matches with the lenght of SOC array 
charging_duration = charge_phase_duration[:len(State_of_Charge_from_charging)]


In [ ]:

function_to_estimate_capacity_in_Ah_from_discharging()

initial_soc_discharge_phase = State_of_Charge_from_charging[-1]  
# Subtract normalized discharged charge (absolute value since discharge current is negative)
State_of_Charge_from_disccharging = initial_soc_discharge_phase - np.abs(cumulative_Ah_discharge_phase) / estimated_capacity_charge_phase_in_Ah 
# Insert initial SOC at beginning for plotting. This is the SoC after the charging phase
State_of_Charge_from_disccharging = np.insert(State_of_Charge_from_disccharging, 0, initial_soc_discharge_phase)  
# Match discharge time array length to SOC array length
discharge_phase_duration_raw = discharge_phase_duration_raw[:len(State_of_Charge_from_disccharging)]  
# We use raw time for discharge plotting (no offset)
time_discharge_phase_combined = discharge_phase_duration_raw 

In [ ]:
# --- Plot SoC and current ---

fig, ax1 = plt.subplots(figsize=(15, 8))  # Create figure and axis

# Plot SOC (in percentage) on left y-axis
line1, = ax1.plot(charging_duration, State_of_Charge_from_charging * 100, label='Charging SoC', color='green')
line2, = ax1.plot(time_discharge_phase_combined, State_of_Charge_from_disccharging * 100, label='Discharging SOC', color='red')

ax1.set_xlabel('Time (s)')     # X-axis label
ax1.set_ylabel('SoC (%)')      # Left Y-axis label for SOC
ax1.set_ylim([0, 105])         # Set SOC limits slightly above 100%
ax1.grid(True)                 # Enable grid

# Plot current on right y-axis with some transparency
ax2 = ax1.twinx()  # Create second y-axis sharing same x-axis
line3, = ax2.plot(time_in_sec, current_in_amperes, label='Current (A)', color='blue', alpha=0.3)

ax2.set_ylabel('Current (A)')   # Right Y-axis label

# Combine legends from both axes
lines = [line1, line2, line3]
labels = [line.get_label() for line in lines]

# Place legend in upper left, slightly lower to avoid overlap
ax1.legend(lines, labels, loc='upper left', bbox_to_anchor=(0.01, 0.95))
plot_title = 'Battery Charge/Discharge Cycle'
plt.title(f'{plot_title} - SoC and Current Profile')  # Plot title
plt.tight_layout()                                 # Adjust layout for neatness
plt.show()                                         # Display plot

# --- Print summary of estimated charge and discharge capacity ---
print(f"Estimated Capacity from charging phase: {estimated_capacity_charge_phase_in_Ah:.3f} Ah")
print(f"Total Discharged capacity from discharge phase: {np.abs(cumulative_Ah_discharge_phase[-1]):.3f} Ah")



